In [ ]:
from oltk.dataset import VaultDataset
from oltk.features import extract_header_props

I personally do not own an obsidian grapth big enough to check certain features.
I do not own any rights on the repository and it's contents by any means, the following data is in full posession of it's rightful author [Luke Tianpei](https://github.com/TianpeiLuke)

In [2]:
!mkdir ./tmp_data
%cd ./tmp_data
!git clone https://github.com/TianpeiLuke/SelfStudyNotes
%cd ../

/home/igkh/codes/ObsidianLTK/examples/tmp_data
Cloning into 'SelfStudyNotes'...
remote: Enumerating objects: 28898, done.
remote: Counting objects: 100% (340/340), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 28898 (delta 325), reused 324 (delta 324), pack-reused 28558 (from 2)
Receiving objects: 100% (28898/28898), 397.54 MiB | 8.68 MiB/s, done.
Resolving deltas: 100% (21018/21018), done.
Updating files: 100% (6492/6492), done.


In [8]:
vault = VaultDataset("./tmp_data/SelfStudyNotes")

In [9]:
%%time
vault.construct()

CPU times: user 502 ms, sys: 61.8 ms, total: 563 ms
Wall time: 545 ms


In [10]:
%%time
samples = vault.get_v()

Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
CPU times: user 19.1 ms, sys: 29.6 ms, total: 48.8 ms
Wall time: 51.7 ms


In [ ]:
from oltk.utils.re_constants import JINJA_EXPR, TEMPLATER_EXPR
import re
import polars as pl

In [12]:
def tidy_up_template(content: str) -> str:
    content = re.sub(JINJA_EXPR, "__TEMPLATE_IMPUTATION__", content)
    content = re.sub(TEMPLATER_EXPR, "__TEMPLATE_IMPUTATION__", content)
    return content

In [13]:
samples = samples.with_columns(
    pl.col("text")
    .map_elements(
        tidy_up_template,
        return_dtype=str,
    )
    .alias("text_without_templates")
).drop(pl.col("text"))

In [15]:
samples = samples.with_columns(
    pl.col("text_without_templates")
    .map_elements(
        lambda x: extract_header_props(x).get("tags", []),
        return_dtype=pl.List(str),
    )
    .alias("tags")
)

In [ ]:
tags_cnt = (
    samples.select(["tags"])
    .explode("tags")
    .drop_nulls()["tags"]
    .value_counts()
    .sort("count", descending=True)
)

In [ ]:
!rm -rf ./tmp_data